# Complete partially-finished MERGE step of a historical run
The 48 hour walltime limit on Gadi means that it's not possible to finish the MERGE step in a single job for long (> ~60 year) large ensemble runs. I need a better fix in the long run, but for now this notebook continues each MERGE job where it left off

Here I use `region` to fill the incomplete block and then pick up continue writing with `append`

In [1]:
import os
import json
import math
import glob
import zarrtools
import numpy as np
import xarray as xr

In [25]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

walltime = '02:00:00'
cores = 48
memory = '192GB'
cluster = PBSCluster(processes=1,
                     walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),
                                '-l mem='+str(memory),
                                '-P xv83',
                                '-l jobfs=100GB',
                                '-l storage=gdata/xv83+gdata/v14+scratch/v14'],
                     local_directory='$PBS_JOBFS',
                     # env_extra=['export MALLOC_TRIM_THRESHOLD_="0"'],
                     header_skip=["select"])

cluster.scale(jobs=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.24.72:8787/status,
Dashboard: http://10.6.24.72:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.24.72:37443,Workers: 0
Dashboard: http://10.6.24.72:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Change this depending on what `realm` your doing and when it died

In [3]:
realm = 'atmos_daily'
incomplete_block = slice(22484, 22792)

### How does the partially-written data look?

In [26]:
test = xr.open_zarr(
    f'/g/data/xv83/ds0092/CAFE/historical/WIP/c5-d60-pX-hist-19601101-96mem/ZARR/atmos_hybrid_daily.zarr')

In [16]:
test['hght'].isel(time=incomplete_block).isnull().sum().values

array(9580032000)

In [17]:
test['hght'].isel(time=slice(0,incomplete_block.start)).isnull().sum().values

array(0)

### Build the dataset

In [18]:
paths_to_merge = sorted(glob.glob('/g/data/xv83/ds0092/CAFE/historical/WIP/c5-d60-pX-hist-19601101-96mem/ZARR/mem???'))
zarr_path = '/g/data/xv83/ds0092/CAFE/historical/WIP/c5-d60-pX-hist-19601101-96mem/ZARR/'
config_file = '/g/data/xv83/ds0092/CAFE/historical/WIP/c5-d60-pX-hist-19601101-96mem/mem004/zarr_specs_CAFE-f6.json'
number_of_time_blocks = 100

with open(config_file) as json_file:
    specs = json.load(json_file)

In [19]:
zarrify_kwargs = specs[realm].copy()
if 'load_chunks' in zarrify_kwargs:
    zarrify_kwargs.pop('load_chunks')
zarr_name = zarrify_kwargs.pop('zarr_name')
zarrify_kwargs['output_file'] = os.path.join(zarr_path, zarr_name)
done_file = os.path.join(zarr_path, f'{zarr_name}{os.path.extsep}done')

In [20]:
import dask
import zarr

print(f'Merging {zarr_name}...', flush=True)
_open = dask.delayed(xr.open_zarr)
datasets = []
merged_files = []
for path in paths_to_merge:
    input_file = os.path.join(path, zarr_name)
    input_zip_file = os.path.join(path, zarr_name+os.path.extsep+'zip')
    # Preferably use DirectoryStore if available, else ZipStore
    if os.path.exists(input_file):
        datasets.append(_open(input_file, consolidated=True))
        merged_files.append(input_file)
    elif os.path.exists(input_zip_file):
        datasets.append(_open(zarr.ZipStore(input_zip_file), consolidated=True))
        merged_files.append(input_zip_file)
    else:
        raise FileNotFoundError(f'Cannot find {input_file} or {input_zip_file}')
dataset = xr.concat(dask.compute(datasets)[0], dim='ensemble').sortby('ensemble')

Merging atmos_hybrid_daily.zarr...


In [21]:
for var in dataset:
    dataset[var].encoding = {}

### Use `region` to write incomplete block

In [22]:
def _drop_vars(ds, sequence_dim='time'):
    # writing a region means that all the variables MUST have sequence_dim
    to_drop = [v for v in ds.variables if sequence_dim not in ds[v].dims]
    return ds.drop_vars(to_drop)
                      
print(f'Writing {incomplete_block}...', flush=True)
dataset_write = dataset.isel({'time': incomplete_block})
zarrtools.zarrify(_drop_vars(dataset_write),
                  **zarrify_kwargs,
                  region={'time': incomplete_block},
                  compute=True)

Writing slice(22484, 22792, None)...
  This dataset is 0.34294323477661237 TB
  Max chunk size is DELP at 132.890625 MB
  Min chunk size is DELP at 132.890625 MB
  Total number of chunks is 2706


### Complete rest of collection using `append`

In [23]:
def _blocks(l, n):
    """Yield successive n-sized chunks from l"""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
# Loop over time blocks 
BLOCK_DIM = 'time'
time_chunk_size = zarrify_kwargs['save_chunks'][BLOCK_DIM]
N = dataset.sizes[BLOCK_DIM]
block_size = time_chunk_size * math.ceil(N / number_of_time_blocks / time_chunk_size) # round to nearest multiple of time chunks

for idx, block in enumerate(_blocks(range(incomplete_block.stop, N), block_size)):
    print(f'Writing {block} of {N}...', flush=True)
    dataset_write = dataset.isel({BLOCK_DIM: slice(block.start, block.stop)})
    zarrtools.zarrify(dataset_write,
                      **zarrify_kwargs,
                      append_dim=BLOCK_DIM,
                      compute=True)

Writing range(22792, 23100) of 29220...
  This dataset is 0.34294333374418784 TB
  Max chunk size is DELP at 132.890625 MB
  Min chunk size is DELP at 132.890625 MB
  Total number of chunks is 2706
Writing range(23100, 23408) of 29220...
  This dataset is 0.34294333374418784 TB
  Max chunk size is DELP at 132.890625 MB
  Min chunk size is DELP at 132.890625 MB
  Total number of chunks is 2706
Writing range(23408, 23716) of 29220...
  This dataset is 0.34294333374418784 TB
  Max chunk size is DELP at 132.890625 MB
  Min chunk size is DELP at 132.890625 MB
  Total number of chunks is 2706
Writing range(23716, 24024) of 29220...
  This dataset is 0.34294333374418784 TB
  Max chunk size is DELP at 132.890625 MB
  Min chunk size is DELP at 132.890625 MB
  Total number of chunks is 2706
Writing range(24024, 24332) of 29220...
  This dataset is 0.34294333374418784 TB
  Max chunk size is DELP at 132.890625 MB
  Min chunk size is DELP at 132.890625 MB
  Total number of chunks is 2706
Writing ra

In [27]:
test['hght'].isnull().sum().values

array(0)

In [28]:
cluster.close()
client.close()